# 01 - Exploración de Datos con Azure ML

Este notebook demuestra el uso de GitHub Copilot para acelerar la exploración de datos en Azure ML.

**Objetivos:**
- Conectar a Azure ML workspace
- Cargar datos desde Azure Data Lake
- Realizar análisis exploratorio (EDA)
- Generar visualizaciones interactivas
- Detectar calidad de datos

In [ ]:
# Imports y configuración
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from great_expectations.dataset import PandasDataset

# Configuración de visualización
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Imports completados")

## 1. Conexión a Azure ML Workspace

In [ ]:
# Conectar a Azure ML
credential = DefaultAzureCredential()
ml_client = MLClient(
    credential=credential,
    subscription_id="<YOUR_SUBSCRIPTION_ID>",
    resource_group_name="<YOUR_RESOURCE_GROUP>",
    workspace_name="<YOUR_WORKSPACE>"
)

print(f"Conectado a workspace: {ml_client.workspace_name}")

## 2. Carga de Datos

💡 **Copilot tip:** Usa `/explain` para entender cómo cargar datos desde diferentes fuentes.

In [ ]:
# Ejemplo: cargar dataset de ejemplo (reemplazar con tus datos)
# Para demo usaremos un dataset sintético
from sklearn.datasets import make_classification

X, y = make_classification(
    n_samples=10000,
    n_features=20,
    n_informative=15,
    n_redundant=5,
    random_state=42
)

# Crear DataFrame
feature_names = [f'feature_{i}' for i in range(X.shape[1])]
df = pd.DataFrame(X, columns=feature_names)
df['target'] = y

print(f"Dataset cargado: {df.shape[0]:,} filas, {df.shape[1]} columnas")
df.head()

## 3. Análisis Exploratorio de Datos (EDA)

In [ ]:
# Resumen estadístico
print("📊 Resumen Estadístico:")
display(df.describe())

# Información del dataset
print("\n📋 Información del Dataset:")
df.info()

In [ ]:
# Análisis de valores nulos
null_counts = df.isnull().sum()
if null_counts.sum() > 0:
    print("⚠️  Valores nulos detectados:")
    print(null_counts[null_counts > 0])
else:
    print("✅ No hay valores nulos")

In [ ]:
# Distribución de la variable target
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Conteo
df['target'].value_counts().plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'])
axes[0].set_title('Distribución de Target', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Clase')
axes[0].set_ylabel('Frecuencia')

# Porcentaje
df['target'].value_counts(normalize=True).plot(kind='pie', ax=axes[1], autopct='%1.1f%%')
axes[1].set_title('Proporción de Clases', fontsize=14, fontweight='bold')
axes[1].set_ylabel('')

plt.tight_layout()
plt.show()

## 4. Análisis de Correlaciones

In [ ]:
# Matriz de correlación
correlation_matrix = df.corr()

# Top correlaciones con target
target_corr = correlation_matrix['target'].sort_values(ascending=False)
print("🎯 Top 10 features correlacionadas con target:")
print(target_corr.head(11))  # 11 porque incluye la correlación consigo mismo

In [ ]:
# Heatmap de correlaciones
plt.figure(figsize=(16, 12))
sns.heatmap(
    correlation_matrix,
    cmap='coolwarm',
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.8}
)
plt.title('Matriz de Correlación de Features', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

## 5. Validación de Calidad con Great Expectations

💡 **Copilot tip:** Usa `/doc` para generar expectativas automáticamente.

In [ ]:
# Crear dataset con Great Expectations
ge_df = PandasDataset(df)

# Definir expectativas
expectations = [
    ge_df.expect_table_row_count_to_be_between(min_value=1000, max_value=100000),
    ge_df.expect_column_values_to_not_be_null('target'),
    ge_df.expect_column_values_to_be_in_set('target', [0, 1]),
]

# Validar
for exp in expectations:
    if exp['success']:
        print(f"✅ {exp['expectation_type']}")
    else:
        print(f"❌ {exp['expectation_type']}: {exp.get('result', 'Failed')}")

## 6. Resumen y Próximos Pasos

**Hallazgos clave:**
- Dataset balanceado con ~50% de cada clase
- Sin valores nulos
- Features con diferentes niveles de correlación con target

**Próximos pasos:**
1. Preparación de datos (notebook 02)
2. Feature engineering
3. Entrenamiento de modelos

In [ ]:
# Guardar dataset procesado para el siguiente notebook
output_path = './data/exploracion_completa.parquet'
df.to_parquet(output_path, index=False)
print(f"✅ Dataset guardado en: {output_path}")